In [1]:
import pandas as pd
import numpy as np
import requests
from collections import Counter

In [2]:
core = 'http://dnd5eapi.co/api/'
pages = ['Ability-Scores', 'Classes', 'Conditions', 'Damage-Types',
         'Equipment-Categories', 'Equipment', 'Features', 'Languages',
         'Levels', 'Magic-Schools','Monsters', 'Proficiencies', 'Races',
         'Skills', 'Spellcasting', 'Spells', 'StartingEquipment',
         'Subclasses', 'Subraces', 'Test', 'Traits', 'Weapon-Properties']

def REQUEST(url):
    try:
        m = requests.get(url, timeout=1)
        m1 = m.json()
        return m1
    except:
        return None        

In [3]:
active = []
for page in pages:
    url = ''.join((core,page))
    try:
        m = requests.get(url, timeout=1)
        m1 = m.json()
        active.append(page)
    except:
        pass

In [4]:
print(active)

['Ability-Scores', 'Classes', 'Conditions', 'Damage-Types', 'Equipment-Categories', 'Equipment', 'Features', 'Languages', 'Magic-Schools', 'Monsters', 'Proficiencies', 'Races', 'Skills', 'Spellcasting', 'Spells', 'StartingEquipment', 'Subclasses', 'Subraces', 'Traits', 'Weapon-Properties']


# Monster Table

In [5]:
r = requests.get(''.join((core,'Monsters')))
x = r.json()
monsters = x['results']
monster_df = pd.DataFrame(monsters)
monster_df.head()

name                                    url
0               Aboleth  http://www.dnd5eapi.co/api/monsters/1
1               Acolyte  http://www.dnd5eapi.co/api/monsters/2
2    Adult Black Dragon  http://www.dnd5eapi.co/api/monsters/3
3  Adult Blue Dracolich  http://www.dnd5eapi.co/api/monsters/4
4     Adult Blue Dragon  http://www.dnd5eapi.co/api/monsters/5

In [6]:
def extractCR(url):
    try:
        m = requests.get(url, timeout=1)
        m1 = m.json()
        return m1['challenge_rating']
    except:
        return(None)

monster_df['CR']= monster_df['url'].apply(extractCR)


In [15]:
#monster_df.set_index('ID',inplace=True)

In [8]:
monster_df.tail()

name                                      url     CR
320   Young Green Dragon  http://www.dnd5eapi.co/api/monsters/321   8.00
321     Young Red Dragon  http://www.dnd5eapi.co/api/monsters/322  10.00
322  Young Silver Dragon  http://www.dnd5eapi.co/api/monsters/323   9.00
323   Young White Dragon  http://www.dnd5eapi.co/api/monsters/324   6.00
324               Zombie  http://www.dnd5eapi.co/api/monsters/325   0.25

In [10]:
def extractKeys(url):
    try:
        m = requests.get(url, timeout=1)
        m1 = m.json()
        return [m1.keys()]
    except:
        return(None)
    
stats = monster_df['url'].apply(extractKeys)

In [11]:
statlist = stats.tolist()

In [12]:

present = []
for entry in statlist:
    present.append(Counter(list(entry[0])))

stat_cnt = pd.DataFrame(present)
stat_cnt.head()

_id  acrobatics  actions  alignment  arcana  armor_class  athletics  \
0    1         NaN      1.0          1     NaN            1        NaN   
1    1         NaN      1.0          1     NaN            1        NaN   
2    1         NaN      1.0          1     NaN            1        NaN   
3    1         NaN      1.0          1     NaN            1        NaN   
4    1         NaN      1.0          1     NaN            1        NaN   

   challenge_rating  charisma  charisma_save     ...       speed  stealth  \
0                 1       NaN            NaN     ...           1      NaN   
1                 1       1.0            NaN     ...           1      NaN   
2                 1       1.0            1.0     ...           1      1.0   
3                 1       1.0            NaN     ...           1      NaN   
4                 1       1.0            1.0     ...           1      1.0   

   strength  strength_save  subtype  survival  type  url  wisdom  wisdom_save  
0         1            NaN        1       NaN     1    1       1          1.0  
1         1            NaN        1       NaN     1    1       1          NaN  
2         1            NaN        1       NaN     1    1       1          1.0  
3         1            NaN        1       NaN     1    1       1          NaN  
4         1            NaN        1       NaN     1    1       1          1.0  

[5 rows x 51 columns]

In [13]:
allstats = stat_cnt.isnull().sum().sort_values()

In [47]:
def SummonMonster(url):
    try:
        m = requests.get(url, timeout=1)
        m1 = m.json()
        return m1
    except:
        return(None)
    
class Monster:
    def __init__(self,url):
        self.stats = SummonMonster(url)
        for key in self.stats:
            if key not in allstats:
                self.stats[key] = 0
        self.condition = []
        
m1 = str(np.random.randint(1,200))
Mon1 = Monster(''.join(('http://www.dnd5eapi.co/api/monsters/',m1)))
print(Mon1.stats['name'])

m2 = str(np.random.randint(1,200))
Mon2 = Monster(''.join(('http://www.dnd5eapi.co/api/monsters/',m2)))
print(Mon2.stats['name'])


Centaur
Elk


In [48]:
def Attack(action, opp):
    name, hit, dmg, bon = action
    dice = dmg.split(' + ')
    damage = 0
    for i in dice:
        n, d = map(int,i.split('d'))
        damage += sum(np.random.randint(1,d,n)) + bon
    roll = hit + np.random.randint(1,20)
    
    if roll >= opp.stats['armor_class']:
        d = opp.stats['hit_points']
        opp.stats['hit_points'] = d-damage  
        return(name,damage)
    else:
        return('miss',0)

def ChooseAction(char):
    valid = False
    options = char.stats['actions']
    while not valid:
        choice = options[np.random.randint(0,len(options)-1)]
        if choice['attack_bonus'] != 0: valid = True
    
    return [choice['name'],choice.get('attack_bonus',0),
            choice.get('damage_dice','0d0'),choice.get('damage_bonus',0)]

def Describe(p1,p2,result):
    name1 = p1.stats['name']
    name2 = p2.stats['name']
    
    print('{0:s} attacks {1:s} with a {2:s} for {3:d} damage'\
              .format(name1,name2,result[0],result[1]))
    print('{0:s} now at {1:d}HP'.format(name2,int(p2.stats['hit_points'])))


### Simple 1 v 1 comabt testing
Randomly selects a combat action from the monster's list of actions and executes. Basic functions above will be expanded out to include conditions in this order
- Crit hit and miss
- Conditions 
     - Advantage and Disadvantage
     - General conditions 
- Spells
     


In [49]:
while Mon1.stats['hit_points'] >0 and Mon2.stats['hit_points'] >0:
    action = ChooseAction(Mon1)
    result = Attack(action, Mon2)
    Describe(Mon1,Mon2,result)
    action = ChooseAction(Mon2)
    result = Attack(action, Mon1)
    Describe(Mon2,Mon1,result)
    print()

45
13
Centaur attacks Elk with a Hooves for 14 damage
Elk now at -1HP


KeyboardInterrupt: 

# PC/NPC generator from classes option

In [260]:
stat_priority = {'Barbarian':['STR','CON'],
                 'Bard': ['CHA','DEX'],
                'Cleric':['WIS','CON','STR'],
                'Druid':['wis','con'],
                'Fighter':['str','dex','con'],
                'Monk':['dex','wis'],
                'Paladin':['str','cha'],
                'Ranger':['dex','wis'],
                'Rogue':['dex','cha'],
                'Sorcerer':['cha','con'],
                'Warlock':['cha','con'],
                'Wizard':['int','dex','con']}

mod = np.arange(-5,11,1)
abss = np.arange(1,32,2)
print(dict(zip(abss,mod)))


{1: -5, 3: -4, 5: -3, 7: -2, 9: -1, 11: 0, 13: 1, 15: 2, 17: 3, 19: 4, 21: 5, 23: 6, 25: 7, 27: 8, 29: 9, 31: 10}


In [250]:
def CleanDict(dic, exclude):
    for ex in exclude:
        dic.pop(ex, None)
    return dic
                
class NPC():
   
    def __init__(self,Class, level):
        self.stats = REQUEST(''.join((core,'classes/',class_dic[Class])))
        exclude = ['_id','index','class_levels','url']
        #removes useless keys
        for ex in exclude:
            self.stats.pop(ex, None)
        
        #simplifies weapon proficiency and saving throw dict
        self.stats['proficiencies'] = [x['name'] for x in self.stats['proficiencies']]
        self.stats['saving_throws']= [x['name'] for x in self.stats['saving_throws']]
        
        #selects random proficiencies
        skill_num = self.stats['proficiency_choices'][0]['choose']
        self.stats['proficiency_choices'] = self.stats['proficiency_choices'][0]['from']
        self.stats['proficiency_choices'] = [x['name'].split(':')[-1].strip() for x in self.stats['proficiency_choices']]
        skill_idx = np.random.randint(0,len(self.stats['proficiency_choices']),skill_num)
        self.stats['Skills'] = [self.stats['proficiency_choices'][x] for x in skill_idx]  
    
        #Sets Starting Equipment
        exclude = ['_id','url','index','class']
        Set = CleanDict(REQUEST(self.stats['starting_equipment']['url']),exclude)                
       
        self.equip = {}
        for i in Set['starting_equipment']:
            item = i['item']
            item = CleanDict(REQUEST(item['url']),exclude)
            self.equip[item.pop('name')]= item

        for c in range(1,Set['choices_to_make']+1):
            #selects and flattens dictionary of choices
            choice = Set['_'.join(('choice',str(c)))]
            choice = [x['from'] for x in choice]
            choice = [x for sublist in choice for x in sublist]
            choice = [x['item'] for x in choice]
            
            #selects a random weapon
            select = np.random.randint(0,len(choice))
            item = choice[select]
            self.equip[item['name']] = CleanDict(REQUEST(item['url']),exclude)
        
        self.stats.pop('starting_equipment',None)
        
        #rolls base stats from equipment
        abilities = ['STR','DEX','CON','WIS','INT','CHA']
        rolls = sorted([sum(np.random.randint(1,7,3)) for i in range(7)])
        top = stat_priority[self.stats['name']]
        for t in top:
            self.stats[t]=rolls.pop()
            abilities.remove(t)
        
        for a in abilities:
            np.random.shuffle(rolls)
            self.stats[a] = rolls.pop()
            

In [252]:
John = NPC('Bard',0)
John.stats

{'CHA': 16,
 'CON': 10,
 'DEX': 13,
 'INT': 9,
 'STR': 9,
 'Skills': ['Insight', 'Persuasion', 'Performance'],
 'WIS': 10,
 'hit_die': 8,
 'name': 'Bard',
 'proficiencies': ['Light armor',
  'Simple weapons',
  'Longswords',
  'Rapiers',
  'Shortswords',
  'Crossbows, hand'],
 'proficiency_choices': ['Acrobatics',
  'Animal Handling',
  'Arcana',
  'Athletics',
  'Deception',
  'History',
  'Insight',
  'Intimidation',
  'Investigation',
  'Medicine',
  'Nature',
  'Perception',
  'Performance',
  'Persuasion',
  'Religion',
  'Sleight of Hand',
  'Stealth',
  'Survival'],
 'saving_throws': ['DEX', 'CHA'],
 'spellcasting': {'class': 'Bard',
  'url': 'http://www.dnd5eapi.co/api/spellcasting/1'},
 'subclasses': [{'name': 'Lore',
   'url': 'http://www.dnd5eapi.co/api/subclasses/2'}]}

In [119]:
REQUEST('http://www.dnd5eapi.co/api/equipment/39')

{'_id': '5a52bca1559f00418e53357f',
 'armor_category': 'Light',
 'armor_class': {'base': 11, 'dex_bonus': True, 'max_bonus': None},
 'cost': {'quantity': 10, 'unit': 'gp'},
 'equipment_category': 'Armor',
 'index': 39,
 'name': 'Leather',
 'stealth_disadvantage': False,
 'str_minimum': 0,
 'url': 'http://www.dnd5eapi.co/api/equipment/39',
 'weight': 10}

In [202]:
REQUEST('http://www.dnd5eapi.co/api/Ability-Scores/')

{'count': 6,
 'results': [{'name': 'STR',
   'url': 'http://www.dnd5eapi.co/api/ability-scores/1'},
  {'name': 'DEX', 'url': 'http://www.dnd5eapi.co/api/ability-scores/2'},
  {'name': 'CON', 'url': 'http://www.dnd5eapi.co/api/ability-scores/3'},
  {'name': 'INT', 'url': 'http://www.dnd5eapi.co/api/ability-scores/4'},
  {'name': 'WIS', 'url': 'http://www.dnd5eapi.co/api/ability-scores/5'},
  {'name': 'CHA', 'url': 'http://www.dnd5eapi.co/api/ability-scores/6'}]}